# geoDB Access

The geoDB is a service provided by the [Euro Data Cube project](https://eurodatacube.com) (EDC) as a payed service. It comes
with a Python client that provides hugh level acess to your data and a certain amount of space in a PostGreSQL database. For managing your data you will need
a management (read/write) account to your database which you can purchase at the [EDC market place](https://hub.eox.at).

You can access the service in two ways:

- By using the Jupyter Python notebook provided by EOX (configuartion free, ```geodb = GeoDBClient()```)
- By using you own Jupyter notebook or Python script by providing a client id and secret to the GeoDBClient (```geodb = GeoDBClient(client_id="myid", client_secret="mysecet")```)

The client ID and secret is also provided by EOX in the latter case. You will find them in your EOX hub account section. You can also provide the credentials
via system environment varibles (GEODB_AUTH_CLIENT_ID and GEODB_AUTH_CLIENT_SECRET). These variables can be supplied via a .env file. 

There are two different types of geoDB accounts: a read only and a management (read/write) access. The system will determine your access right through your 
authentication credentials.



## Manage Collections in your GeoDB



### Connecting to the GeoDB

In [6]:
from xcube_geodb.core.geodb import GeoDBClient

In [8]:
# Use accordingly
# geodb = GeoDBClient(client_id="your id from EOX", client_secret="your secret from EOX")
geodb = GeoDBClient()

In [11]:
# If you are logged in, this will tell you what account the system currently uses
geodb.whoami

'geodb_admin'

In [14]:
# Lets get already existing collections
ds = geodb.get_collections()
ds

,table_name
0,land_use
1,test
2,test2


### Creating collections

Once the connection has been established you will be able to create a collection. The collection will contain standard properties (fields) plus custom properties
which you can add at your disgretion. Please use [PostGreSQL type definitions](https://www.postgresql.org/docs/11/datatype.html). We recommend stying simple with
your data types as we have not tested every single type.

In [16]:
# Have a look at fiona feature schema
collections = {
        "land_use": 
        {
            "crs": 3794,
            "properties": 
            {
                "RABA_PID": "float", 
                "RABA_ID": "float", 
                "D_OD": "date"
            }
        }
    }


geodb.create_collections(collections)

GeoDBError: {"hint":null,"details":null,"code":"42P07","message":"relation \"geodb_admin_land_use\" already exists"}

In [17]:
ds = geodb.get_collections()
ds

,table_name
0,land_use
1,test
2,test2


### Loading data into a dataset

Once the table has been created, you can load data into the dataset. The example below loads a shapefile. The attributes of the shapefile correspond to the dataset's properties.


In [18]:
import geopandas
gdf = geopandas.read_file('data/sample/land_use.shp')
gdf

,RABA_PID,RABA_ID,D_OD,geometry
0,4770326.0,1410,2019-03-26,"POLYGON ((453952.629 91124.177, 453952.696 911..."
1,4770325.0,1300,2019-03-26,"POLYGON ((453810.376 91150.199, 453812.552 911..."
2,2305689.0,7000,2019-02-25,"POLYGON ((456099.635 97696.070, 456112.810 976..."
3,2305596.0,1100,2019-02-25,"POLYGON ((455929.405 97963.785, 455933.284 979..."
4,2310160.0,1100,2019-03-11,"POLYGON ((461561.512 96119.256, 461632.114 960..."
...,...,...,...,...
9822,6253989.0,1600,2019-03-08,"POLYGON ((460637.334 96865.891, 460647.927 969..."
9823,6252044.0,1600,2019-03-26,"POLYGON ((459467.868 96839.686, 459467.770 968..."
9824,6245985.0,2000,2019-04-08,"POLYGON ((459488.998 94066.248, 459498.145 940..."
9825,6245986.0,2000,2019-02-20,"POLYGON ((459676.680 94000.000, 459672.469 939..."


In [19]:
geodb.insert_into_collection('land_use', gdf)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



Data inserted into land_use

In [20]:
geodb.get_collection('land_use', query="raba_id=eq.7000")

,id,created_at,modified_at,geometry,raba_pid,raba_id,d_od
0,10194,2020-03-27T07:43:29.792941+00:00,None,"POLYGON ((461024.210 100486.472, 461021.110 10...",225585,7000,2019-04-06
1,10408,2020-03-27T07:43:29.792941+00:00,None,"POLYGON ((459905.731 93486.400, 459901.642 934...",2358628,7000,2019-02-20
2,10655,2020-03-27T07:43:29.792941+00:00,None,"POLYGON ((461378.981 99411.298, 461378.981 994...",210612,7000,2019-03-27
3,10656,2020-03-27T07:43:29.792941+00:00,None,"POLYGON ((462448.912 100391.708, 462446.507 10...",210613,7000,2019-03-21
4,10195,2020-03-27T07:43:29.792941+00:00,None,"POLYGON ((459610.699 100660.079, 459612.119 10...",225587,7000,2019-04-06
...,...,...,...,...,...,...,...
379,19246,2020-03-27T07:43:29.792941+00:00,None,"POLYGON ((456523.670 94000.000, 456550.368 939...",6187993,7000,2019-03-13
380,19373,2020-03-27T07:43:29.792941+00:00,None,"POLYGON ((460793.279 94000.000, 460797.466 939...",6219403,7000,2019-02-21
381,19387,2020-03-27T07:43:29.792941+00:00,None,"POLYGON ((461750.000 98434.292, 461750.000 984...",6219602,7000,2019-03-20
382,19451,2020-03-27T07:43:29.792941+00:00,None,"POLYGON ((463628.912 97501.218, 463636.859 975...",6215237,7000,2019-03-20


### Delete from a Collection

In [21]:
geodb.delete_from_collection('land_use', query="raba_id=eq.7000")

Data from land_use deleted

In [22]:
geodb.get_collection('land_use', query="raba_id=eq.7000")

,Empty Result


### Updating a Collection

In [23]:
geodb.get_collection('land_use', query="raba_id=eq.1300")

,id,created_at,modified_at,geometry,raba_pid,raba_id,d_od
0,10,2020-01-31T11:08:35.241344+00:00,2020-01-31T11:09:11.84153+00:00,"POLYGON ((456547.427 91543.640, 456544.255 915...",2318555,1300,2000-01-01
1,1540,2020-01-31T11:08:35.241344+00:00,2020-01-31T11:09:11.84153+00:00,"POLYGON ((461553.191 92037.359, 461555.607 920...",4643601,1300,2000-01-01
2,2,2020-01-31T11:08:35.241344+00:00,2020-01-31T11:09:11.84153+00:00,"POLYGON ((453810.376 91150.199, 453812.552 911...",4770325,1300,2000-01-01
3,63,2020-01-31T11:08:35.241344+00:00,2020-01-31T11:09:11.84153+00:00,"POLYGON ((456201.531 98685.274, 456199.109 986...",2304287,1300,2000-01-01
4,86,2020-01-31T11:08:35.241344+00:00,2020-01-31T11:09:11.84153+00:00,"POLYGON ((454709.766 97354.278, 454704.878 973...",2331038,1300,2000-01-01
...,...,...,...,...,...,...,...
1785,19593,2020-03-27T07:43:29.792941+00:00,None,"POLYGON ((458247.583 99746.065, 458245.202 997...",6268059,1300,2019-03-04
1786,19594,2020-03-27T07:43:29.792941+00:00,None,"POLYGON ((461932.283 90520.703, 461931.619 905...",6263561,1300,2019-04-01
1787,19625,2020-03-27T07:43:29.792941+00:00,None,"POLYGON ((457231.170 90902.256, 457234.861 909...",6264862,1300,2019-01-05
1788,19626,2020-03-27T07:43:29.792941+00:00,None,"POLYGON ((461378.845 91124.318, 461490.224 911...",6264865,1300,2019-02-21


In [24]:
geodb.update_collection('land_use', query="raba_id=eq.1300", values={'d_od': '2000-01-01'})

land_use updated

In [25]:
geodb.get_collection('land_use', query="raba_id=eq.1300")

,id,created_at,modified_at,geometry,raba_pid,raba_id,d_od
0,10,2020-01-31T11:08:35.241344+00:00,2020-03-27T07:44:10.11363+00:00,"POLYGON ((456547.427 91543.640, 456544.255 915...",2318555,1300,2000-01-01
1,2,2020-01-31T11:08:35.241344+00:00,2020-03-27T07:44:10.11363+00:00,"POLYGON ((453810.376 91150.199, 453812.552 911...",4770325,1300,2000-01-01
2,5944,2020-01-31T11:08:35.241344+00:00,2020-03-27T07:44:10.11363+00:00,"POLYGON ((463488.069 90719.797, 463489.977 907...",5955535,1300,2000-01-01
3,395,2020-01-31T11:08:35.241344+00:00,2020-03-27T07:44:10.11363+00:00,"POLYGON ((460018.736 96059.962, 460025.012 960...",6302716,1300,2000-01-01
4,396,2020-01-31T11:08:35.241344+00:00,2020-03-27T07:44:10.11363+00:00,"POLYGON ((459978.716 96047.034, 459980.975 960...",6302715,1300,2000-01-01
...,...,...,...,...,...,...,...
1785,19592,2020-03-27T07:43:29.792941+00:00,2020-03-27T07:44:10.11363+00:00,"POLYGON ((458304.555 100146.103, 458306.726 10...",6268058,1300,2000-01-01
1786,19593,2020-03-27T07:43:29.792941+00:00,2020-03-27T07:44:10.11363+00:00,"POLYGON ((458247.583 99746.065, 458245.202 997...",6268059,1300,2000-01-01
1787,19594,2020-03-27T07:43:29.792941+00:00,2020-03-27T07:44:10.11363+00:00,"POLYGON ((461932.283 90520.703, 461931.619 905...",6263561,1300,2000-01-01
1788,19625,2020-03-27T07:43:29.792941+00:00,2020-03-27T07:44:10.11363+00:00,"POLYGON ((457231.170 90902.256, 457234.861 909...",6264862,1300,2000-01-01


### Managing Properties of a Collection

In [26]:
geodb.get_collections()

,table_name
0,land_use
1,test
2,test2


In [27]:
geodb.get_properties('land_use')

,table_name,column_name,data_type
0,land_use,id,integer
1,land_use,created_at,timestamp with time zone
2,land_use,modified_at,timestamp with time zone
3,land_use,geometry,USER-DEFINED
4,land_use,raba_pid,double precision
5,land_use,raba_id,double precision
6,land_use,d_od,date


In [36]:
geodb.add_property('land_use', "testä_prop", 'integer')

Properties added

In [37]:
geodb.get_properties('land_use')

,table_name,column_name,data_type
0,land_use,id,integer
1,land_use,created_at,timestamp with time zone
2,land_use,modified_at,timestamp with time zone
3,land_use,geometry,USER-DEFINED
4,land_use,raba_pid,double precision
5,land_use,raba_id,double precision
6,land_use,d_od,date
7,land_use,testä_prop,integer


In [30]:
geodb.drop_property('land_use', 'test_prop')

Properties ['test_prop'] dropped from land_use

In [31]:
geodb.get_properties('land_use')

,table_name,column_name,data_type
0,land_use,id,integer
1,land_use,created_at,timestamp with time zone
2,land_use,modified_at,timestamp with time zone
3,land_use,geometry,USER-DEFINED
4,land_use,raba_pid,double precision
5,land_use,raba_id,double precision
6,land_use,d_od,date


In [32]:
geodb.add_properties('land_use', properties={'test1': 'integer', 'test2': 'date'})

Properties added

In [33]:
geodb.get_properties('land_use')

,table_name,column_name,data_type
0,land_use,id,integer
1,land_use,created_at,timestamp with time zone
2,land_use,modified_at,timestamp with time zone
3,land_use,geometry,USER-DEFINED
4,land_use,raba_pid,double precision
5,land_use,raba_id,double precision
6,land_use,d_od,date
7,land_use,test1,integer
8,land_use,test2,date


In [34]:
geodb.drop_properties('land_use', properties=['test1', 'test2'])

Properties ['test1', 'test2'] dropped from land_use

In [35]:
geodb.get_properties('land_use')

,table_name,column_name,data_type
0,land_use,id,integer
1,land_use,created_at,timestamp with time zone
2,land_use,modified_at,timestamp with time zone
3,land_use,geometry,USER-DEFINED
4,land_use,raba_pid,double precision
5,land_use,raba_id,double precision
6,land_use,d_od,date
